In [34]:
import pandas as pd
import glob
import os
import pytz

In [35]:
Glacier='Wolverine'
Station='990'

In [40]:
if Glacier + Station== "Gulkana1480":
    par_dir=r"Q:\Project Data\GlacierData\Benchmark_Program\Data/" +Glacier+ r'/AllYears/Wx/Raw/telemeteredNWIS'
if Glacier + Station =="Wolverine990":
    par_dir=r"Q:\Project Data\GlacierData\Benchmark_Program\Data\Wolverine\AllYears\Wx\Raw\telemeteredNWIS\Historical_1995to2017"

In [41]:
os.chdir(par_dir)
fls=glob.glob('*.csv')

In [42]:
#Remove the summary file if already made previously
fls.remove("NWIS_data_"+Glacier + ".csv")

In [43]:
alldat=pd.DataFrame()
for fl in fls:
    dat=pd.read_csv(fl, header=14)
    var_name=fl.replace(".csv", "")
    print(var_name)
    dat=dat[['ISO 8601 UTC', ' Value']]
    dat.rename(columns = {' Value': var_name}, inplace = True)
    if fl==fls[0]:
        alldat=alldat.append(dat)
        continue
    alldat=alldat.merge(dat, on='ISO 8601 UTC', how='outer')

StageCumulative
TAspirated1
TAspirated2
Tpassive1
Tpassive2
WindDir
WindGustSpeed
WindSpeed


In [44]:
alldat=alldat.rename(columns={'ISO 8601 UTC': "UTC_time"})

In [45]:
alldat.head()

,UTC_time,StageCumulative,TAspirated1,TAspirated2,Tpassive1,Tpassive2,WindDir,WindGustSpeed,WindSpeed
0,1995-09-21T20:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1995-09-21T20:15:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1995-09-21T20:30:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1995-09-21T20:45:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,1995-09-22T00:00:00Z,4572.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [46]:
#Convert wind gust speed from mph to m/s!! (this is only @ Wolv 990)
if Glacier + Station == 'Wolverine990':
    alldat.WindGustSpeed=alldat.WindGustSpeed * 0.44704 #convert from mph to m/s

In [47]:
alldat['UTC_time']=pd.to_datetime(alldat.UTC_time, format='%Y-%m-%dT%H:%M:%SZ')

In [48]:
#Set timezone of known UTC column
alldat['UTC_time'].timezone='UTC'
#Create column for local time
timezone='America/Anchorage'
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

alldat=alldat.set_index('UTC_time')
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

#Create local time
alldat['local_time'] = alldat.index.tz_localize('UTC').tz_convert(local_timezone)

In [49]:
#Remove UTC from index
alldat=alldat.reset_index()

In [50]:
alldat['UTC_time']=alldat.UTC_time.dt.strftime('%Y/%m/%d %H:%M') #set as string in desired format
alldat['local_time']=alldat.local_time.dt.strftime('%Y/%m/%d %H:%M')
alldat.to_csv("NWIS_data_" + Glacier+ Station +".csv", index=False, float_format='%g')